In [1]:
import pyodbc
import pandas as pd
import numpy as np

In [2]:
connection = pyodbc.connect(
    "DRIVER={SQL Server};"
    "SERVER=advancementreporting.win.louisiana.edu;"
    "DATABASE=CRM_Advance;"
    "Trusted_Connection=yes;"
)

In [3]:
query = """SELECT 
      [elcn_PrimaryID]
      ,[donor_sortname]
      ,[donor_constituent_type]
      ,[campaign_const_type_selected_reason]
      ,[primary_prospect_manager]
      ,[elcn_ContributionNumber]
      ,[designation_elcn_code]
      ,[elcn_contributiondate]
      ,[drive_fy]
      ,[production]
      ,[cash_receipts]
      ,[household_production]
      ,[household_cash]
      ,[elcn_amount]
      ,[elcn_jointcontribution]
	  ,annual_fund_bucket
  FROM [CRM_Advance].[dbo].[annual_fund_giving]
  where annual_fund_bucket = 'Alumni Loyalty';"""

print(query)

SELECT 
      [elcn_PrimaryID]
      ,[donor_sortname]
      ,[donor_constituent_type]
      ,[campaign_const_type_selected_reason]
      ,[primary_prospect_manager]
      ,[elcn_ContributionNumber]
      ,[designation_elcn_code]
      ,[elcn_contributiondate]
      ,[drive_fy]
      ,[production]
      ,[cash_receipts]
      ,[household_production]
      ,[household_cash]
      ,[elcn_amount]
      ,[elcn_jointcontribution]
	  ,annual_fund_bucket
  FROM [CRM_Advance].[dbo].[annual_fund_giving]
  where annual_fund_bucket = 'Alumni Loyalty';


In [4]:
donor_df = pd.read_sql(query, connection)
donor_df.to_csv('donors.csv', index=False)

C:\Users\C00580017\AppData\Local\Temp\ipykernel_20384\211791463.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  donor_df = pd.read_sql(query, connection)


In [5]:
donor_df['elcn_contributiondate'] = pd.to_datetime(donor_df['elcn_contributiondate'])
donor_df['drive_fy'] = donor_df['drive_fy'].astype(int)

Creating target Variables

In [6]:
donor_df = donor_df.sort_values(by=['elcn_PrimaryID','drive_fy'])
donor_df['Will_Donate_Next_Year'] = donor_df.groupby('elcn_PrimaryID')['drive_fy'].shift(-1).notna().astype(int)


In [7]:
donor_df

,elcn_PrimaryID,donor_sortname,donor_constituent_type,campaign_const_type_selected_reason,primary_prospect_manager,elcn_ContributionNumber,designation_elcn_code,elcn_contributiondate,drive_fy,production,cash_receipts,household_production,household_cash,elcn_amount,elcn_jointcontribution,annual_fund_bucket,Will_Donate_Next_Year
107519,10000,"Ward, Laura O.",Alumnus/Alumna,"primary.no spouse, no sc",None,280624,LOYALTY,2007-06-11 10:59:00,2007,0.0,0.0,100.0,100.0,0.0,True,Alumni Loyalty,1
3032,10000,"Ward, Laura O.",Alumnus/Alumna,"primary.no spouse, no sc",None,329872,LOYALTY,2011-09-28 10:59:00,2012,0.0,0.0,60.0,60.0,0.0,True,Alumni Loyalty,0
109480,100013,"Lewis, Wilfred J.",Alumnus/Alumna,"primary.no spouse, no sc",None,132827,LOYALTY,1992-05-08 10:59:00,1992,30.0,30.0,30.0,30.0,30.0,False,Alumni Loyalty,1
21087,100013,"Lewis, Wilfred J.",Alumnus/Alumna,primary=spouse,None,216910,LOYALTY,2000-06-15 10:59:00,2000,40.0,40.0,40.0,40.0,40.0,True,Alumni Loyalty,0
208643,100015,"Lewis, Evelyn T.",Alumnus/Alumna,"primary.no spouse, no sc",None,292864,LOYALTY,2008-09-16 10:59:00,2009,40.0,40.0,40.0,40.0,40.0,False,Alumni Loyalty,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29368,9995,"Ward, Gerald D.",Alumnus/Alumna,"primary.no spouse, no sc",None,114759,LOYALTY,1990-09-30 10:59:00,1991,25.0,25.0,25.0,25.0,25.0,False,Alumni Loyalty,0
139616,99951,"Lewis, Nancy L.",Alumnus/Alumna,"primary.no spouse, no sc",None,264675,LOYALTY,2005-10-11 10:59:00,2006,0.0,0.0,0.0,0.0,0.0,True,Alumni Loyalty,0
16988,99968,"Lewis, Ida B.",Alumnus/Alumna,"primary.no spouse, no sc",None,73678,LOYALTY,1987-06-18 10:59:00,1987,25.0,25.0,25.0,25.0,25.0,False,Alumni Loyalty,0
20508,99973,"Wagstaff, Chevelle L.",Alumnus/Alumna,"primary.no spouse, no sc",None,191321,LOYALTY,1997-11-17 10:59:00,1998,30.0,30.0,30.0,30.0,30.0,False,Alumni Loyalty,0


In [8]:
donor_df['Total_Donations'] = donor_df.groupby('elcn_PrimaryID')['cash_receipts'].transform('count')
donor_df['Total_Amount_Donated'] = donor_df.groupby('elcn_PrimaryID')['cash_receipts'].transform('sum')
donor_df['Average_Donation_Amount'] = donor_df['Total_Amount_Donated'] / donor_df['Total_Donations']
donor_df['drive_fy'] = donor_df['drive_fy'].astype(int)
donor_df['Recency'] = donor_df.groupby('elcn_PrimaryID')['drive_fy'].transform(lambda x: x.max() - x)

In [9]:
donor_df = pd.get_dummies(donor_df, columns=['donor_constituent_type','campaign_const_type_selected_reason','primary_prospect_manager','designation_elcn_code','annual_fund_bucket'])

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

In [17]:
# Split data
X = donor_df.drop(columns=['Will_Donate_Next_Year','elcn_PrimaryID', 'donor_sortname', 'elcn_ContributionNumber','elcn_contributiondate'])
y = donor_df['Will_Donate_Next_Year']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
categorical_columns = X_train.select_dtypes(include=['object']).columns
print(categorical_columns)

Index([], dtype='object')


In [19]:
print(X_train.dtypes)

drive_fy                                                               int64
production                                                           float64
cash_receipts                                                        float64
household_production                                                 float64
household_cash                                                       float64
elcn_amount                                                          float64
elcn_jointcontribution                                                  bool
Total_Donations                                                        int64
Total_Amount_Donated                                                 float64
Average_Donation_Amount                                              float64
Recency                                                                int64
donor_constituent_type_Alumnus/Alumna                                   bool
donor_constituent_type_Corporate Foundation                             bool

In [20]:
# Train model
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [21]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
print("ROC-AUC Score:", roc_auc_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.92      0.92      6187
           1       0.99      0.98      0.99     37010

    accuracy                           0.98     43197
   macro avg       0.95      0.95      0.95     43197
weighted avg       0.98      0.98      0.98     43197

ROC-AUC Score: 0.9534277950817464
